In [1]:
import keras_tuner
import numpy as np
import sklearn.metrics
import tensorflow as tf
import Modules.constants as constants
import Modules.ds_loader as ds_loader

SAMPLE_PERCENTAGE = 1.0
DATA_PATH = constants.DATASET
TRAIN_DIR = DATA_PATH / "train"
VAL_DIR = DATA_PATH / "val"
TEST_DIR = DATA_PATH / "test"

X_train, y_train = ds_loader.load_all_data(TRAIN_DIR, SAMPLE_PERCENTAGE)
X_val, y_val = ds_loader.load_all_data(VAL_DIR, SAMPLE_PERCENTAGE)
X_test, y_test = ds_loader.load_all_data(TEST_DIR, SAMPLE_PERCENTAGE)

2025-04-02 10:41:39.158436: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-02 10:41:39.168162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743583299.178252  222705 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743583299.181158  222705 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743583299.190242  222705 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# 1-D convolutional ResNet model 
# https://pmc.ncbi.nlm.nih.gov/articles/PMC10128986/#sec012
class Resnet(keras_tuner.HyperModel):
    def residual_block(self, inputs, c_units, kernel_size):
        shortcut = tf.keras.layers.Conv1D(filters=c_units, kernel_size=1, padding='same')(inputs)
        
        x = tf.keras.layers.Conv1D(filters=c_units, kernel_size=kernel_size, strides=1, padding='same')(inputs)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        x = tf.keras.layers.Conv1D(filters=c_units, kernel_size=kernel_size, strides=1, padding='same')(x)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        # SKIP CONNECTION
        x = tf.keras.layers.Add()([x, shortcut]) 
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.MaxPooling1D(pool_size=5, strides=2)(x)
        
        return x

    def build(self, hp):
        c_units = hp.Int('c_units', min_value=8, max_value=128, step=2)
        d_units = hp.Int('d_units', min_value=8, max_value=128, step=2)
        dropout = hp.Float('dropout', min_value = 0.4, max_value=0.5)
        lr = hp.Float('lr', min_value=0.00001, max_value=0.001)
        n = hp.Choice('#_res_layers', [1,2,3,4])
        k_units = hp.Int('k_units', min_value=3, max_value=15, step=1)
        m = hp.Float('momentum', min_value=0.1, max_value=0.9, step=0.05)
        # INPUT LAYER
        inputs = tf.keras.Input(shape=(500,12))
        
        # RESIDUALS
        x = self.residual_block(inputs, c_units, kernel_size=k_units)
        for _ in range(n):  
            x = self.residual_block(x, c_units, kernel_size=k_units)

        # CLASSIFIER
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(d_units, activation='relu')(x)
        x = tf.keras.layers.Dropout(dropout)(x)  

        x = tf.keras.layers.Dense(d_units //2, activation='relu')(x)
        x = tf.keras.layers.Dropout(dropout)(x)  

        # OUTPUT
        outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

        model = tf.keras.Model(inputs, outputs)
        
        model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=m),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"]
        )
        
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            epochs = hp.Int("epochs", 10, 200, 10),
            batch_size= hp.Choice("batch_size", [16, 32, 64]),
            *args,
            **kwargs,
        )

In [3]:
tuner = keras_tuner.RandomSearch(
    Resnet(),
    max_trials=100,
    objective='val_loss',
    directory="Results/RESNET",
    project_name="ECGClassification01",
    )
tuner.search_space_summary()

Reloading Tuner from Results/RESNET/ECGClassification01/tuner0.json
Search space summary
Default search space size: 8
c_units (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 32, 'step': 8, 'sampling': 'linear'}
d_units (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 64, 'step': 8, 'sampling': 'linear'}
dropout (Float)
{'default': 0.3, 'conditions': [], 'min_value': 0.3, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
lr (Float)
{'default': 1e-06, 'conditions': [], 'min_value': 1e-06, 'max_value': 0.01, 'step': None, 'sampling': 'linear'}
#_res_layers (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
k_units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 9, 'step': 1, 'sampling': 'linear'}
epochs (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 10, 'sampling': 'linear'}
batch_size (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64], '

In [4]:
models = tuner.get_best_models(num_models=1)
best_model = models[0]
best_model.summary()
best_model.save("Results/BEST_RESNET_00.h5") 

I0000 00:00:1743583303.492876  222705 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2532 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
/home/capitan/.venv/tenv/lib64/python3.11/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 110 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 500, 12)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 500, 8)    │        296 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 500, 8)    │          0 │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 500, 8)    │         32 │ re_lu[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 500, 8)    │        200 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 500, 8)    │          0 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 500, 8)    │         32 │ re_lu_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 500, 8)    │        104 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 500, 8)    │          0 │ batch_normalizat… │
│                     │                   │            │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 500, 8)    │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 500, 8)    │         32 │ re_lu_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 248, 8)    │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 248, 8)    │        200 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 248, 8)    │          0 │ conv1d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 248, 8)    │         32 │ re_lu_3[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 248, 8)    │        200 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_4 (ReLU)      │ (None, 248, 8)    │          0 │ conv1d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 248, 8)    │         32 │ re_lu_4[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 248, 8)    │         72 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 248, 8)    │          0 │ batch_normalizat… │
│                     │                   │            │ conv1d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 14,476 (56.55 KB)

 Trainable params: 14,284 (55.80 KB)

 Non-trainable params: 192 (768.00 B)

In [5]:
tuner.results_summary()

Results summary
Results in Results/RESNET/ECGClassification01
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 15 summary
Hyperparameters:
c_units: 8
d_units: 48
dropout: 0.4
lr: 0.005436814517384435
#_res_layers: 3
k_units: 3
epochs: 60
batch_size: 16
momentum: 0.1
Score: 0.23586633801460266

Trial 25 summary
Hyperparameters:
c_units: 32
d_units: 40
dropout: 0.3
lr: 0.006893850234946141
#_res_layers: 3
k_units: 5
epochs: 90
batch_size: 16
Score: 0.24849657714366913

Trial 09 summary
Hyperparameters:
c_units: 32
d_units: 40
dropout: 0.5
lr: 0.005220392976469985
#_res_layers: 3
k_units: 9
epochs: 90
batch_size: 16
Score: 0.26381364464759827

Trial 07 summary
Hyperparameters:
c_units: 24
d_units: 16
dropout: 0.3
lr: 0.0004544534662388181
#_res_layers: 3
k_units: 9
epochs: 100
batch_size: 32
Score: 0.3774159848690033

Trial 03 summary
Hyperparameters:
c_units: 8
d_units: 8
dropout: 0.4
lr: 0.0016475816682847081
#_res_layers: 3
k_units: 8
epochs: 100
batch_size: 32

In [6]:
model = tf.keras.models.load_model("Results/BEST_RESNET_03.h5")

In [7]:
params = tuner.get_best_hyperparameters(num_trials=1)[0]
print(params.values)

{'c_units': 8, 'd_units': 48, 'dropout': 0.4, 'lr': 0.005436814517384435, '#_res_layers': 3, 'k_units': 3, 'epochs': 60, 'batch_size': 16, 'momentum': 0.1}


In [8]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=1000)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

I0000 00:00:1743583304.667137  222922 service.cc:152] XLA service 0x7f2b0c004e60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743583304.667155  222922 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-04-02 10:41:44.680980: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
E0000 00:00:1743583304.773782  222922 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1743583304.785798  222922 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but s

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib64/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib64/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib64/python3.11/asyncio/events.py", line 84, in _run

  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/ipykernel/ke

In [ ]:
y_pred = model.predict(X_test)

if y_pred.shape[1] == 1:  
    y_pred_binary = (y_pred > 0.5).astype(int)
    auc = sklearn.metrics.roc_auc_score(y_test, y_pred)  
else:
    y_pred_binary = np.argmax(y_pred, axis=1)  
    auc = sklearn.metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

print("Classification Report (Test Data):")
print(sklearn.metrics.classification_report(y_test, y_pred_binary))
print(f"AUC: {auc}")

y_train_pred = model.predict(X_train)
y_train_pred = np.argmax(y_train_pred, axis=1)

print("\nClassification Report (Train Data):")
print(sklearn.metrics.classification_report(y_train, y_train_pred))


17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step
Classification Report (Test Data):
              precision    recall  f1-score   support

           0       0.86      0.80      0.83       111
           1       0.84      0.90      0.87       115
           2       0.98      0.98      0.98       195
           3       0.92      0.92      0.92       111

    accuracy                           0.91       532
   macro avg       0.90      0.90      0.90       532
weighted avg       0.91      0.91      0.91       532

AUC: 0.9817942208013336
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step

Classification Report (Train Data):
              precision    recall  f1-score   support

           0       0.90      0.93      0.91      1774
           1       0.97      0.91      0.94      1840
           2       0.98      0.99      0.99      3110
           3       0.95      0.96      0.95      1778

    accuracy                           0.95      8502
   macro avg       0.95      0.95      0.95      8502
weighted